# Open Library SQLite

We're going to take some [Open Library dumps](https://openlibrary.org/developers/dumps) for authors, works and editions and put them into SQLite.
Because they're so big (several GB of compressed text) they can't fit into memory and SQLite gives an easy way to manage large files.

In [1]:
from pathlib import Path

import gzip
import json
import sqlite3

from tqdm.auto import tqdm

This assumes the dumps are downloaded into `data_path` and are from `ol_dump_date`.

A better way could be to stream the download; the files are [here](https://archive.org/details/ol_exports?sort=-publicdate) (for example https://archive.org/download/ol_dump_2022-06-06/ol_dump_editions_2022-06-06.txt.gz) and `_latest` redirects to the latest version (e.g. https://openlibrary.org/data/ol_dump_editions_latest.txt.gz).
There's also a torrent version which may be less resource intensive for the Internet Archive.

In [2]:
ol_dump_date = '2022-06-06'
data_path = Path('../data/01_raw')

def ol_path(segment):
    return data_path / f'ol_dump_{segment}_{ol_dump_date}.txt.gz'

def ol_data(segment):
    with gzip.open(ol_path(segment), 'rt') as f:
        for line in f:
            yield tuple(line.split('\t', 5))

We'll write the items to the database in [batches](https://skeptric.com/python-minibatching/) of ~1,000 to reduce the overhead of SQLite parsing the `INSERT STATEMENT`

In [3]:
def minibatch(seq, size):
    items = []
    for x in seq:
        items.append(x)
        if len(items) >= size:
            yield items
            items = []
    if items:
        yield items

This puts it all together:

* Create a table for the data
* Start a transaction
* Read the data dump from disk
* Insert into SQLite 1,000 rows at a time.

In [4]:
def create_segment(con, segment, batch_size=1_000):
    con.execute(f'CREATE TABLE {segment} (type TEXT, key TEXT, revision INT, last_modified TEXT, json TEXT);')
    with con:
        for batch in minibatch(tqdm(ol_data(segment)), batch_size):
            con.executemany(f'INSERT INTO {segment} VALUES (?,?,?,?,?)', batch)

Let's create our database

In [5]:
con = sqlite3.connect('../data/01_raw/openlibrary.sqlite')

We'll set some options to make writes faster, which are safe in this one-off bulk import scenario (see [this blog](http://blog.quibb.org/2010/08/fast-bulk-inserts-into-sqlite/) and [Stackoverflow post](https://stackoverflow.com/questions/1711631/improve-insert-per-second-performance-of-sqlite) for details).

In [6]:
con.execute("PRAGMA synchronous=OFF")
con.execute("PRAGMA count_changes=OFF")
con.execute("PRAGMA journal_mode=MEMORY")
con.execute("PRAGMA temp_store=MEMORY")

Now let's create all the tables

In [7]:
%time create_segment(con, 'authors')

0it [00:00, ?it/s]

CPU times: user 36.5 s, sys: 3.12 s, total: 39.6 s
Wall time: 40.9 s


In [8]:
%time create_segment(con, 'works')

0it [00:00, ?it/s]

CPU times: user 2min 18s, sys: 15.8 s, total: 2min 34s
Wall time: 2min 54s


In [9]:
%time create_segment(con, 'editions')

0it [00:00, ?it/s]

CPU times: user 5min 19s, sys: 38.1 s, total: 5min 57s
Wall time: 7min 7s


And close off the connection

In [10]:
con.close()